In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../Data/Raw/Gold_Price_data.csv')

In [3]:
data.head(3)

,Date,Price,Open,High,Low,Volume,Chg%
0,2014-01-01,29542,29435,29598,29340,2930,0.25
1,2014-01-02,29975,29678,30050,29678,3140,1.47
2,2014-01-03,29727,30031,30125,29539,3050,-0.83


In [4]:
data['lag_P'] = data[data.columns[1]].shift(3)
data['lag_O'] = data[data.columns[2]].shift(3)
data['lag_H'] = data[data.columns[3]].shift(3)
data['lag_L'] = data[data.columns[4]].shift(3)
data['lag_V'] = data[data.columns[5]].shift(3)
data['lag_C'] = data[data.columns[6]].shift(3)
data = data.dropna(axis = 0, how = 'any')
data.reset_index(drop = True, inplace = True)
data.drop(columns = {'Open', 'High', 'Low', 'Volume', 'Chg%'}, inplace = True)
data.head(3)

,Date,Price,lag_P,lag_O,lag_H,lag_L,lag_V,lag_C
0,2014-01-04,29279,29542.0,29435.0,29598.0,29340.0,2930.0,0.25
1,2014-01-06,29119,29975.0,29678.0,30050.0,29678.0,3140.0,1.47
2,2014-01-07,28959,29727.0,30031.0,30125.0,29539.0,3050.0,-0.83


In [5]:
def time_features(df, date_columns):
    
    df['Year'] = pd.to_datetime(df[date_columns]).dt.year
    df['Month'] = pd.to_datetime(df[date_columns]).dt.month
    df['Day'] = pd.to_datetime(df[date_columns]).dt.day
    df['Weekday'] = pd.to_datetime(df[date_columns]).dt.weekday
    
    return df

data = time_features(data, 'Date')

In [6]:
columns ={'Price', 'lag_P', 'lag_O', 'lag_H', 'lag_L', 'lag_V', 'lag_C',
       'Year', 'Month', 'Day', 'Weekday'}

def minmax(data):
    for col in columns:
        scaler = MinMaxScaler(feature_range = (0,1))
        data[col] = scaler.fit_transform(data[col].values.reshape(-1,1))
    return data

Gold_data = minmax(data)

In [7]:
Gold_data.head()

,Date,Price,lag_P,lag_O,lag_H,lag_L,lag_V,lag_C,Year,Month,Day,Weekday
0,2014-01-04,0.149943,0.158273,0.152732,0.155756,0.157452,0.027404,0.552305,0.0,0.0,0.100000,0.833333
1,2014-01-06,0.144875,0.171988,0.160382,0.169941,0.168380,0.029368,0.660461,0.0,0.0,0.166667,0.000000
2,2014-01-07,0.139807,0.164133,0.171493,0.172295,0.163886,0.028526,0.456560,0.0,0.0,0.200000,0.166667
3,2014-01-08,0.139016,0.149943,0.147822,0.145744,0.155480,0.000000,0.396277,0.0,0.0,0.233333,0.333333
4,2014-01-09,0.141011,0.144875,0.148483,0.149385,0.148109,0.228021,0.481383,0.0,0.0,0.266667,0.500000


In [8]:
#Gold_data.to_csv('../Data/Processed/Gold_data.csv', index = False)